In [43]:
import pandas as pd
import re
from tqdm import tqdm
import time

original_hour = 'https://transparency.entsog.eu/api/v1/operationaldatas.csv?pointDirection={}&from=2019-10-01&to=2020-04-01&indicator=Physical%20Flow&periodType=hour&timezone=CET&limit=-1&dataset=1'
original_day='https://transparency.entsog.eu/api/v1/operationaldatas.csv?pointDirection={}&from=2020-01-01&to=2020-10-13&indicator=Physical%20Flow&periodType=day&timezone=CET&limit=-1&dataset=1'
firm_interrupt_day = 'https://transparency.entsog.eu/api/v1/operationalData.csv?pointDirection={}&from=2020-01-01&to=2020-10-13&indicator=Firm%20Technical,Interruptible%20Total&periodType=day&timezone=CET&limit=-1&dataset=1'
firm_interrupt_hour='https://transparency.entsog.eu/api/v1/operationalData.csv?pointDirection={}&from=2020-01-01&to=2020-10-13&indicator=Firm%20Technical,Interruptible%20Total&periodType=hour&timezone=CET&limit=-1&dataset=1'
firm_day='https://transparency.entsog.eu/api/v1/operationalData.csv?pointDirection={}&from=2020-01-01&to=2020-10-13&indicator=Firm%20Technical&periodType=day&timezone=CET&limit=-1&dataset=1'
firm_hour='https://transparency.entsog.eu/api/v1/operationalData.csv?pointDirection={}&from=2020-01-01&to=2020-10-13&indicator=Firm%20Technical&periodType=hour&timezone=CET&limit=-1&dataset=1'
#kWh/h
# or kWh/d
#need divide 10e6

+ pysical day
+ pysical hour
+ firm day
+ firm hour
+ firm+interrupt day
+ firm+interrupt hour

## load direction meta data

In [5]:
#load data

operationaldata=pd.read_csv('https://transparency.entsog.eu/api/v1/Interconnections.csv?limit=-1')
fullpointdata=pd.read_csv('https://transparency.entsog.eu/api/v1/connectionpoints.csv?limit=-1')
direction=pd.read_csv('https://transparency.entsog.eu/api/v1/operatorpointdirections.csv?limit=-1')

# only EU
simple_direction_dataset = direction[direction.tpEURelationship == 'within EU'][['id', 'pointKey',
                                                                                 'pointLabel', 'directionKey',
                                                                                 'relatedPoints', 'tsoItemIdentifier',
                                                                                 'tpTsoCountry', 'adjacentCountry',
                                                                                 'tpEURelationship', 'tpCrossBorderPointType',
                                                                                 'tpTsoGCVMax']].copy()
# clean id ,remove some strange things in string
simple_direction_dataset.id = simple_direction_dataset.id.str[1:].str.lower()

# what id looks like
# de-tso-0009itp-00047exit


# solve id not so clean problem

simple_direction_dataset.id = simple_direction_dataset.id.str.replace(
    'exit', 'split')
simple_direction_dataset.id = simple_direction_dataset.id.str.replace(
    'entry', 'split')
simple_direction_dataset.id = simple_direction_dataset.id.str.split(
    'split').str[0]
simple_direction_dataset.id = simple_direction_dataset.id + \
    simple_direction_dataset.directionKey

# limit to crossborder
p = simple_direction_dataset[simple_direction_dataset.adjacentCountry !=
                             simple_direction_dataset.tpTsoCountry]
p.head(1)

,id,pointKey,pointLabel,directionKey,relatedPoints,tsoItemIdentifier,tpTsoCountry,adjacentCountry,tpEURelationship,tpCrossBorderPointType,tpTsoGCVMax
0,at-sso-0001ugs-00273entry,UGS-00273,Haiming 2 7F/bn,entry,NaN,NaN,AT,DE,within EU,Cross-Border EU|EU,NaN


# problem 1


In [6]:
p[p.tsoItemIdentifier=='21Z000000000272M']

#this one suppose to be a storage infrastructure, but it is crossborad so ?
# can be seen in website

,id,pointKey,pointLabel,directionKey,relatedPoints,tsoItemIdentifier,tpTsoCountry,adjacentCountry,tpEURelationship,tpCrossBorderPointType,tpTsoGCVMax
70,de-tso-0009ugs-00050entry,UGS-00050,Haiming 2 7F /OGE,entry,Haiming 2 7F,21Z000000000272M,DE,AT,within EU,Cross-Border EU|EU,NaN
71,de-tso-0009ugs-00050exit,UGS-00050,Haiming 2 7F /OGE,exit,Haiming 2 7F,21Z000000000272M,DE,AT,within EU,Cross-Border EU|EU,NaN
81,de-tso-0010ugs-00273entry,UGS-00273,Haiming 2 7F/bn,entry,Haiming 2 7F,21Z000000000272M,DE,AT,within EU,Cross-Border EU|EU,NaN
82,de-tso-0010ugs-00273exit,UGS-00273,Haiming 2 7F/bn,exit,Haiming 2 7F,21Z000000000272M,DE,AT,within EU,Cross-Border EU|EU,NaN


# remove this kind of storage !!!!!
# firm and interrupt definition
    how many firm> physical
    if smaller change the pipe it self
# physical flow per hour

In [8]:
pd.read_csv(original_day.format('de-tso-0009ugs-00050entry')).head()

,id,dataSet,indicator,periodType,periodFrom,periodTo,operatorKey,tsoEicCode,operatorLabel,pointKey,...,restorationInformation,capacityType,capacityBookingStatus,isCamRelevant,isNA,originalPeriodFrom,isCmpRelevant,bookingPlatformKey,bookingPlatformLabel,bookingPlatformURL
0,1Physical FlowdayDE-TSO-0009UGS-00050entrykWh/...,1,Physical Flow,day,2019-01-01 08:00,2019-01-02 08:00,DE-TSO-0009,21X-DE-C-A0A0A-T,Open Grid Europe,UGS-00050,...,NaN,NaN,NaN,NaN,1,2018-10-09T08:00:00+02:00,NaN,NaN,NaN,NaN
1,1Physical FlowdayDE-TSO-0009UGS-00050entrykWh/...,1,Physical Flow,day,2019-01-02 08:00,2019-01-03 08:00,DE-TSO-0009,21X-DE-C-A0A0A-T,Open Grid Europe,UGS-00050,...,NaN,NaN,NaN,NaN,1,2018-10-09T08:00:00+02:00,NaN,NaN,NaN,NaN
2,1Physical FlowdayDE-TSO-0009UGS-00050entrykWh/...,1,Physical Flow,day,2019-01-03 08:00,2019-01-04 08:00,DE-TSO-0009,21X-DE-C-A0A0A-T,Open Grid Europe,UGS-00050,...,NaN,NaN,NaN,NaN,1,2018-10-09T08:00:00+02:00,NaN,NaN,NaN,NaN
3,1Physical FlowdayDE-TSO-0009UGS-00050entrykWh/...,1,Physical Flow,day,2019-01-04 08:00,2019-01-05 08:00,DE-TSO-0009,21X-DE-C-A0A0A-T,Open Grid Europe,UGS-00050,...,NaN,NaN,NaN,NaN,1,2018-10-09T08:00:00+02:00,NaN,NaN,NaN,NaN
4,1Physical FlowdayDE-TSO-0009UGS-00050entrykWh/...,1,Physical Flow,day,2019-01-05 08:00,2019-01-06 08:00,DE-TSO-0009,21X-DE-C-A0A0A-T,Open Grid Europe,UGS-00050,...,NaN,NaN,NaN,NaN,1,2018-10-09T08:00:00+02:00,NaN,NaN,NaN,NaN


# problem2:
Pipe in Pipe

-----------

# start compare

## load compare data 

In [9]:
# compare data from lisa
compare_data = pd.read_excel('Capacities_ENTSOG_2020.xlsx')
compare_data.set_index('Unnamed: 0', inplace=True)
m=compare_data.stack().reset_index()
m.columns=['A','B','V']
m
# GWh/d

,A,B,V
0,AT,DE,390.0
1,AT,HU,153.0
2,AT,IT,1149.0
3,AT,SI,113.0
4,AT,SK,247.0
...,...,...,...
77,TR,GR,49.0
78,UA,HU,517.0
79,UA,SK,250.0
80,UK,BE,652.0


## Physical day data

In [44]:
#get capcity
 
answer=[]
for i in tqdm(p.id):
    trytime=0
    while trytime<2:
        try:
            url=original_hour.format(i)
            #get data
            temp_table=pd.read_csv(url)
            #united id 
            temp_table.id=i
            #get max Physically flow
            temp_table=temp_table[['id','value']].groupby('id').max()
            #set index
            temp_table.reset_index(inplace=True)
            
            temp_table.to_csv('tmp_table/tmp_table_{}.csv'.format(i))
            answer.append(temp_table)
            break
        except:
            trytime+=1
           # if trytime==2:
           #     print(i,'not successful')
                
    

100%|██████████| 444/444 [13:08<00:00,  1.78s/it]


In [45]:
physical_capacity=pd.concat(answer)


physical_capacity=p.merge(physical_capacity,left_on='id',right_on='id')[['directionKey','tpTsoCountry','adjacentCountry','value']]

def fix_direction(dataframe):
    if dataframe['directionKey']=='entry':
        temp_value=dataframe['tpTsoCountry']
        dataframe['tpTsoCountry']=dataframe['adjacentCountry']
        dataframe['adjacentCountry']=temp_value
    return dataframe

physical_capacity=physical_capacity.apply(fix_direction,axis=1)

physical_capacity.drop('directionKey',axis=1,inplace=True)


physical_capacity=physical_capacity.groupby(by=['tpTsoCountry','adjacentCountry']).sum().reset_index()

#from kWh/h to GWh/d
physical_capacity.value=physical_capacity.value/10e6

physical_capacity.head()

,tpTsoCountry,adjacentCountry,value
0,AT,DE,9.132193
1,AT,HU,1.412276
2,AT,IT,10.347393
3,AT,SI,0.775748
4,AT,SK,0.000000


In [27]:
physical_capacity.columns=['tpTsoCountry','adjacentCountry','physical_value']

# only firm capacity daily 

In [26]:
#get capcity
 
answer=[]
for i in tqdm(p.id):
    trytime=0
    while trytime<2:
        try:
            url=firm_day.format(i)
            #get data
            temp_table=pd.read_csv(url)
            #united id 
            temp_table.id=i
            #get max Physically flow
            temp_table=temp_table[['id','value']].groupby('id').max()
            #set index
            temp_table.reset_index(inplace=True)
            
            temp_table.to_csv('tmp_table/tmp_table_{}.csv'.format(i))
            answer.append(temp_table)
            break
        except:
            trytime+=1
           # if trytime==2:
           #     print(i,'not successful')
            

firm_capacity=pd.concat(answer)


firm_capacity=p.merge(firm_capacity,left_on='id',right_on='id')[['directionKey','tpTsoCountry','adjacentCountry','value']]

def fix_direction(dataframe):
    if dataframe['directionKey']=='entry':
        temp_value=dataframe['tpTsoCountry']
        dataframe['tpTsoCountry']=dataframe['adjacentCountry']
        dataframe['adjacentCountry']=temp_value
    return dataframe

firm_capacity=firm_capacity.apply(fix_direction,axis=1)

firm_capacity.drop('directionKey',axis=1,inplace=True)


firm_capacity=firm_capacity.groupby(by=['tpTsoCountry','adjacentCountry']).sum().reset_index()

#from kWh/h to GWh/d
firm_capacity.value=firm_capacity.value/10e6

firm_capacity.head()

100%|██████████| 444/444 [01:54<00:00,  3.88it/s]


,tpTsoCountry,adjacentCountry,value
0,AT,DE,325.204558
1,AT,HU,30.615840
2,AT,IT,241.360503
3,AT,SI,25.177973
4,AT,SK,123.660800


In [25]:
#check if answer is correct format
a=0
for i in answer:
    a=max(a,len(i))
a

1

In [28]:
firm_capacity.columns=['tpTsoCountry','adjacentCountry','firm_value']

firm capacty+ interruptable capacity
------------

they are different reccords with different labels, just easy to sum up

In [18]:
#firm capcacity and interpretable 
#get capcity

##def change_id(dataframe):
##    if dataframe['indicator']=='Interruptible Total':
##        dataframe['id']=str(dataframe['id'])[24:-45]
##    else:
##        dataframe['id']=str(dataframe['id'])[18:-25]
##    return dataframe


answer_firm=[]
unsuccessful=0
for i in tqdm(p.id):
    trytime=0
    while trytime<2:
        # try no more than 2 time
        try:
            #if first try not successful, wait 2 seconds, inorder to avoid block by server because of too ofen visit
            #if trytime>0:
            #    time.sleep(2)
            url=firm_interrupt_day.format(i)
            #get data
            temp_table=pd.read_csv(url)
            #united id 
            temp_table.id=i
            #get max Physically flow
            temp_table=temp_table.groupby('indicator').max()[['id','value']]
            #set index
            temp_table.reset_index(inplace=True)
            temp_table.drop('indicator',axis=1,inplace=True)
            
            #temp_table.to_csv('tmp_table/tmp_table_{}.csv'.format(i))
            answer_firm.append(temp_table)
            break
        except:
            trytime+=1
            if trytime==2:
                unsuccessful+=1
print(unsuccessful)                
    

100%|██████████| 444/444 [02:24<00:00,  3.08it/s]

179


In [20]:
firm_inter_capacity=pd.concat(answer_firm)
firm_inter_capacity=p.merge(firm_inter_capacity,left_on='id',right_on='id')[['directionKey','tpTsoCountry','adjacentCountry','value']]

def fix_direction(dataframe):
    if dataframe['directionKey']=='entry':
        temp_value=dataframe['tpTsoCountry']
        dataframe['tpTsoCountry']=dataframe['adjacentCountry']
        dataframe['adjacentCountry']=temp_value
    return dataframe

firm_inter_capacity=firm_inter_capacity.apply(fix_direction,axis=1)

firm_inter_capacity.drop('directionKey',axis=1,inplace=True)

firm_inter_capacity=firm_inter_capacity.groupby(by=['tpTsoCountry','adjacentCountry']).sum().reset_index()

#from kWh/h to GWh/d
firm_inter_capacity.value=firm_inter_capacity.value/10e6

firm_inter_capacity.value=firm_inter_capacity.value.round()

In [29]:
firm_inter_capacity.columns=['tpTsoCountry','adjacentCountry','firm_inter_value']

In [30]:
final_daily_capacity=firm_inter_capacity.merge(firm_capacity,how='outer',left_on=['tpTsoCountry','adjacentCountry'],right_on=['tpTsoCountry','adjacentCountry'])
final_daily_capacity=final_daily_capacity.merge(physical_capacity,how='outer',left_on=['tpTsoCountry','adjacentCountry'],right_on=['tpTsoCountry','adjacentCountry'])

In [31]:
final_daily_capacity

,tpTsoCountry,adjacentCountry,firm_inter_value,firm_value,physical_value
0,AT,DE,89.0,325.204558,208.192438
1,AT,SI,28.0,25.177973,15.295789
2,AT,SK,288.0,123.660800,0.889948
3,BE,DE,379.0,330.594278,78.485359
4,BE,FR,258.0,317.586725,141.557777
...,...,...,...,...,...
58,BG,GR,NaN,24.038404,22.092744
59,GR,BG,NaN,12.965220,9.045171
60,HU,AT,NaN,0.000000,0.000000
61,HU,RO,NaN,15.491722,14.684754


In [32]:
def add_compare(dataframe):
    return compare_data[dataframe['adjacentCountry']][dataframe['tpTsoCountry']]
final_daily_capacity['compare_value']=physical_capacity.apply(add_compare,axis=1)

In [35]:
final_daily_capacity
#GWh/d
final_daily_capacity.to_excel('final_compare.xlsx')

In [12]:
407+47+642+340.8

#compare data is daily capacity!

1436.8

In [36]:
import seaborn as sns

In [ ]:
final_daily_capacity

In [ ]:
sns.barplot(x=)

In [ ]:
direction.columns
# 'tpTsoGCVMax', 'tpTsoGCVRemarks', 'tpTsoGCVUnit' are working together, tpTsoGCVMax have Capacity but with lots of  

In [ ]:
direction['tpCrossBorderPointType'].value_counts()

In [ ]:
#direction['tpRmkGridConversionFactorCapacityDefault'].value_counts(dropna=False)
for i in direction.columns:
    if re.search('max',i,re.IGNORECASE):
        print(i)


In [ ]:
# a lot of missing value
direction[direction.tpEURelationship=='within EU'][['tpTsoGCVMax', 'tpTsoGCVRemarks', 'tpTsoGCVUnit']]

In [ ]:
simple_direction_dataset.id

In [ ]:
direction[direction.tpEURelationship=='within EU']['adjacentCountry'].value_counts(dropna=False)

In [ ]:
operationaldata.iloc[3]
#https://transparency.entsog.eu/api/v1/operationalData.xlsx?forceDownload=true&pointDirection=at-sso-0001ugs-00273exit&from=2020-09-14&to=2020-10-13&indicator=Physical%20Flow&periodType=day&timezone=CET&limit=-1&dataset=1&directDownload=true

In [ ]:
direction.iloc[10]

In [ ]:
fullpointdata

In [ ]:
fullpointdata.tpMapY.value_counts(dropna=False)

In [ ]:
# Cross-Border Transmission IP within EU

EU_crossborder=fullpointdata[fullpointdata.tpMapY=='Cross-Border Transmission IP within EU']

In [ ]:
EU_crossborder.iloc[2]

In [ ]:
operationaldata[(operationaldata['fromCountryKey']=='FR')&(operationaldata['toCountryKey']=='DE')][['pointKey','pointLabel','toPointKey','toPointLabel']]

In [ ]:
operationaldata.columns

In [ ]:
operationaldata[operationaldata['toTsoItemIdentifier']=='21Z0000000000406'][['pointKey','pointLabel','toPointKey','toPointLabel']]

In [ ]:
operationaldata[operationaldata['toTsoItemIdentifier']=='37Z000000004972G'][['pointKey','pointLabel','toPointKey','toPointLabel']]

In [37]:
import Visualisierungen
import Utility
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

Visual1 = Visualisierungen.Visual()

EMAP_Raw = Visual1.dataset['EMAP_Raw'].frame_dict
EMAP_Raw_PipeSegments = EMAP_Raw['PipeSegments']

dataset=Visual1.dataset

Loading BokehJS ...

Loading BokehJS ...

Dataset
 ----------------
Status: Waiting input!


Component
 ----------------
Status: Waiting input!



Instruction
---------------
choose dataset
please choose dataset from: 

*********************


 ['EMAP_Raw', 'GIE_Raw', 'IGG', 'INET_Filled', 'INET_Raw', 'LKD_Raw', 'NO_Raw'] 


*********************

Example: use your object.choose_dataset(['EMAP_Raw','GIE_Raw']) 

--------------
choose component
please choose component from:


*********************

 ['BorderPoints', 'Compressors', 'ConnectionPoints', 'Consumers', 'EntryPoints', 'InterConnectionPoints', 'LNGs', 'Nodes', 'PipeLines', 'PipePoints', 'PipeSegments', 'Processes', 'Productions', 'Storages'] 


*********************



Example: use your object.component(['Nodes','Nodes']) 



In [38]:
PipeSegments=dataset['IGG'].frame_dict['PipeSegments']

In [42]:
PipeSegments.columns

Index(['id', 'name', 'source_id', 'node_id', 'lat', 'long', 'country_code',
       'comment', 'param', 'uncertainty', 'method', 'tags', 'diameter_mm',
       'end_year', 'is_H_gas', 'is_bothDirection', 'lat_mean', 'length_km',
       'long_mean', 'max_cap_M_m3_per_d', 'max_pressure_bar', 'num_compressor',
       'start_year', 'start', 'end'],
      dtype='object')

In [39]:
PipeSegments['country_code']=PipeSegments.country_code.apply(Utility.string2list)

In [40]:
PipeSegments['start']=PipeSegments.country_code.apply(lambda x: x[0])
PipeSegments['end']=PipeSegments.country_code.apply(lambda x: x[1])

In [41]:
crossborder=PipeSegments[PipeSegments.start!=PipeSegments.end]
crossborder=crossborder[['start','end','max_cap_M_m3_per_d','length_km']].groupby(['start','end']).sum()
crossborder.reset_index(inplace=True)

In [37]:
EU=['BE','BG','CZ','DK','DE','EE','IE','EL','ES','FR','HR','IT'
    ,'CY','LV','LT','LU','HU','MT','NL','AT','PL','PT','RO','SI','SK','FI','SE','UK','IS','LI','NO','CH']
crossborder[crossborder.start.isin(EU)&crossborder.end.isin(EU)]

# max_cap_M_m3_per_d:Thisisthemaximumannualgasvolumethatthepipecantransmitinunitsof[Gm3/d].

,start,end,max_cap_M_m3_per_d,length_km
1,AT,SI,39.012654,41.384650
3,BE,DE,67.987270,11.361273
4,BE,FR,36.489864,123.474829
6,BE,NL,43.143146,9.036500
8,BG,RO,43.143146,11.174001
11,CH,DE,59.159113,20.317848
12,CH,FR,59.159113,67.833904
13,CH,IT,59.159113,72.193174
14,CZ,DE,41.821494,2.421477
15,CZ,PL,27.390000,8.397890


In [ ]:
# convert list to adjency matrix
adjancecy_matrix=pd.crosstab(m.A,m.B,m.V,aggfunc='sum')
adjancecy_matrix.fillna(0,inplace=True)
adjancecy_matrix